In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping


In [36]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [37]:
#preprocess the data
#drop irrelevant data

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [38]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [39]:
# Encode categorical variable

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [40]:
onehot_encoder_geo = OneHotEncoder()
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [41]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [42]:
#combining columns with the original data
data = pd.concat([data.drop('Geography',axis = 1),geo_encoded_df],axis=1)

In [43]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [44]:
#Divide the dataset into dependent and independent features

X = data.drop('Exited', axis=1)
Y = data['Exited']

In [45]:
#split the data into training and testing sets

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [46]:
# save the encoders and scaler
import pickle
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler,file)

In [47]:
# define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layer = 1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_dim=(X_train.shape[1],)))

    for _ in range(layer - 1):
        model.add(Dense(neurons, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [48]:
# Create a Keras Classifier

model = KerasClassifier(model=create_model,
                           verbose=0, # Keep verbose low during grid search
                           # Default values for hyperparams (optional, but good practice)
                           neurons=32,
                           layer=1)

In [49]:
# Define the Grid Search Parameters

param_grid = {
    'neurons': [16, 32, 64, 128],
    'layer': [1, 2],
    'epochs': [50, 100],
    'batch_size': [32, 64]
}

In [54]:
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=1, # Safer starting point
                    cv=3,
                    verbose=2) # Higher verbosity for GridSearchCV progress

try:
    # Make sure X_train and Y_train are defined before this line!
    if 'X_train' in locals() and 'Y_train' in locals():
      grid_result = grid.fit(X_train, Y_train)
      print("\nGridSearchCV Complete.")
      print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    else:
      print("Error: X_train or Y_train not defined. Please ensure data preprocessing is done.")

except Exception as e:
    print(f"\nAn error occurred during GridSearchCV: {e}")
    import traceback
    traceback.print_exc()


Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] END ......batch_size=32, epochs=50, layer=1, neurons=16; total time=   0.0s
[CV] END ......batch_size=32, epochs=50, layer=1, neurons=16; total time=   0.0s
[CV] END ......batch_size=32, epochs=50, layer=1, neurons=16; total time=   0.0s
[CV] END ......batch_size=32, epochs=50, layer=1, neurons=32; total time=   0.0s
[CV] END ......batch_size=32, epochs=50, layer=1, neurons=32; total time=   0.0s
[CV] END ......batch_size=32, epochs=50, layer=1, neurons=32; total time=   0.0s
[CV] END ......batch_size=32, epochs=50, layer=1, neurons=64; total time=   0.0s
[CV] END ......batch_size=32, epochs=50, layer=1, neurons=64; total time=   0.0s
[CV] END ......batch_size=32, epochs=50, layer=1, neurons=64; total time=   0.0s
[CV] END .....batch_size=32, epochs=50, layer=1, neurons=128; total time=   0.0s
[CV] END .....batch_size=32, epochs=50, layer=1, neurons=128; total time=   0.0s
[CV] END .....batch_size=32, epochs=50, layer=1,

D:\Projects\ANN Classification\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
D:\Projects\ANN Classification\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
D:\Projects\ANN Classification\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regula

[CV] END ......batch_size=64, epochs=50, layer=1, neurons=64; total time=   0.0s
[CV] END ......batch_size=64, epochs=50, layer=1, neurons=64; total time=   0.0s
[CV] END ......batch_size=64, epochs=50, layer=1, neurons=64; total time=   0.0s
[CV] END .....batch_size=64, epochs=50, layer=1, neurons=128; total time=   0.0s
[CV] END .....batch_size=64, epochs=50, layer=1, neurons=128; total time=   0.0s
[CV] END .....batch_size=64, epochs=50, layer=1, neurons=128; total time=   0.0s
[CV] END ......batch_size=64, epochs=50, layer=2, neurons=16; total time=   0.0s
[CV] END ......batch_size=64, epochs=50, layer=2, neurons=16; total time=   0.0s
[CV] END ......batch_size=64, epochs=50, layer=2, neurons=16; total time=   0.0s
[CV] END ......batch_size=64, epochs=50, layer=2, neurons=32; total time=   0.0s
[CV] END ......batch_size=64, epochs=50, layer=2, neurons=32; total time=   0.0s
[CV] END ......batch_size=64, epochs=50, layer=2, neurons=32; total time=   0.0s
[CV] END ......batch_size=64

D:\Projects\ANN Classification\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
D:\Projects\ANN Classification\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
D:\Projects\ANN Classification\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regula